## Libraries

In [90]:
from datetime import datetime
import numpy as np
import pandas as pd
import requests
from enum import Enum

class ActionType(Enum):
    BUY = 1
    DONOTHING = 0
    SELL = -1

#### Function to make an API call to Binance

In [3]:
def make_api_call(base_url, endpoint="", method="GET", **kwargs):
    # Construct the full URL
    full_url = f'{base_url}{endpoint}'

    # Make the API call
    response = requests.request(method=method, url=full_url, **kwargs)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        return response
    else:
        # If the request was not successful, raise an exception with the error message
        raise Exception(f'API request failed with status code {response.status_code}: {response.text}')

## Getting data

In [48]:
def get_binance_historical_data(symbol, interval, start_date, end_date):
    
    # define basic parameters for call
    base_url = 'https://fapi.binance.com'
    endpoint = '/fapi/v1/klines'
    method = 'GET'
    
    # Set the start time parameter in the params dictionary
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': 1500,
        'startTime': start_date, # Start time in milliseconds
        'endTime': end_date # end time in milliseconds
    }


    # Make initial API call to get candles
    response = make_api_call(base_url, endpoint=endpoint, method=method, params=params)
    
    # initalize candles data
    candles_data = []

    # Append the received candles to the list
    candles_data.extend(response.json())

    # Update the start time for the next API call
    params['startTime'] = candles_data[-1][0] + 1 # last candle open_time + 1ms    

    while len(response.json()) > 0:
        # Make the next API call
        response = make_api_call(base_url, endpoint=endpoint, method=method, params=params)

        # Append the received candles to the list
        candles_data.extend(response.json())

        # Update the start time for the next API call
        params['startTime'] = candles_data[-1][0] + 1 # last candle open_time + 1ms
        
        if params['startTime'] > params['endTime']:
            break

            

    
    # Wrap the candles data as a pandas DataFrame
    columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
               'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    dtype={
    'open_time': 'datetime64[ms, Asia/Jerusalem]',
    'open': 'float64',
    'high': 'float64',
    'low': 'float64',
    'close': 'float64',
    'volume': 'float64',
    'close_time': 'datetime64[ms, Asia/Jerusalem]',
    'quote_asset_volume': 'float64',
    'number_of_trades': 'int64',
    'taker_buy_base_asset_volume': 'float64',
    'taker_buy_quote_asset_volume': 'float64',
    'ignore': 'float64'
    }
    
    df = pd.DataFrame(candles_data, columns=columns)
    df = df.astype(dtype)

    return df

In [50]:
symbol = 'BTCUSDT'
interval = '30m'
start_date = int(datetime(year=2023, month=1, day=1).timestamp() * 1000)
end_date = int(datetime(year=2024, month=1, day=1).timestamp() * 1000)

btcusdt_df = get_binance_historical_data(symbol, '30m', start_date, end_date)
btcusdt_df

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,2023-01-01 00:00:00+02:00,16544.0,16565.1,16496.0,16540.7,7737.533,2023-01-01 00:29:59.999000+02:00,1.279086e+08,35451,3338.726,5.520766e+07,0.0
1,2023-01-01 00:30:00+02:00,16540.8,16550.8,16461.8,16515.1,8929.034,2023-01-01 00:59:59.999000+02:00,1.473757e+08,41907,3740.250,6.174233e+07,0.0
2,2023-01-01 01:00:00+02:00,16515.1,16524.6,16483.1,16520.9,5215.713,2023-01-01 01:29:59.999000+02:00,8.608995e+07,27411,2761.688,4.558437e+07,0.0
3,2023-01-01 01:30:00+02:00,16520.9,16546.9,16516.5,16537.6,3127.373,2023-01-01 01:59:59.999000+02:00,5.171514e+07,17078,1772.384,2.930818e+07,0.0
4,2023-01-01 02:00:00+02:00,16537.5,16540.9,16513.4,16539.4,2832.734,2023-01-01 02:29:59.999000+02:00,4.681919e+07,16561,1373.220,2.269722e+07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17516,2023-12-31 22:00:00+02:00,42659.9,42724.5,42600.0,42627.2,2559.181,2023-12-31 22:29:59.999000+02:00,1.091479e+08,28539,1367.677,5.833755e+07,0.0
17517,2023-12-31 22:30:00+02:00,42627.2,42662.5,42543.3,42599.1,2171.755,2023-12-31 22:59:59.999000+02:00,9.251599e+07,27578,1098.223,4.678832e+07,0.0
17518,2023-12-31 23:00:00+02:00,42599.2,42717.0,42586.7,42674.8,1468.758,2023-12-31 23:29:59.999000+02:00,6.265517e+07,20273,862.639,3.679818e+07,0.0
17519,2023-12-31 23:30:00+02:00,42674.8,42689.1,42558.2,42558.9,2325.252,2023-12-31 23:59:59.999000+02:00,9.909701e+07,24200,1099.718,4.687127e+07,0.0


## Helping Functions

In [5]:
def calc_TR(df) -> pd.DataFrame:
    df['prev_close'] = df['close'].shift(1)

    df['TR'] = df.apply(lambda row: max(row['high'], row['prev_close']) - min(row['low'], row['prev_close']), axis=1)
    return df

In [6]:
def calc_ATR(df, atr_length) -> pd.DataFrame:
    df['ATR'] = df['TR'].rolling(window=atr_length).mean()
    return df

In [7]:
def calc_crossover(df, a, b, target_col_name):
    df[target_col_name] = (df[a] > df[b]) & (df[a].shift(1) < df[b].shift(1))
    return df

In [8]:
def calc_UTBot(df, key_value, atr_length):
    df['loss_threshold'] = key_value * df['ATR']
    df['trailing_stop'] = np.nan

    for i in range(1, len(df)):

        # upward trend
        if (df['close'][i] > df['trailing_stop'][i - 1]) & (df['close'][i - 1] > df['trailing_stop'][i - 1]):
            df['trailing_stop'][i] = max(df['trailing_stop'][i - 1], df['close'][i] - df['loss_threshold'][i])

        # downward trend
        elif (df['close'][i] < df['trailing_stop'][i - 1]) & (df['close'][i - 1] < df['trailing_stop'][i - 1]):
            df['trailing_stop'][i] = min(df['trailing_stop'][i - 1], df['close'][i] - df['loss_threshold'][i])

        elif (df['close'][i] > df['trailing_stop'][i - 1]):
            df['trailing_stop'][i] = df['close'][i] - df['loss_threshold'][i]

        else:
            df['trailing_stop'][i] = df['close'][i] + df['loss_threshold'][i]

        # stop loss

    df = calc_crossover(df, 'close', 'trailing_stop', 'crossover_above')
    df = calc_crossover(df, 'trailing_stop', 'close', 'crossover_below')

    df['buy'] = df.crossover_above.astype(int)
    df['sell'] = df.crossover_below.astype(int)
    return df

In [9]:
def EMACalc(df, length, target_col_name):
    df[target_col_name] = df['close'].ewm(span=length,min_periods=length, adjust=False).mean()
    return df

In [10]:
def MacdDiff(df, fast_length, slow_length):
    EMACalc(df, fast_length, 'fastEMA')
    EMACalc(df, slow_length, 'slowEMA')
    df['macd_diff'] = df['fastEMA'] - df['slowEMA']
    return df

In [26]:
df.shape[0]

289

In [92]:
def TR(close: pd.Series, high: pd.Series, low: pd.Series) -> pd.Series:
    return (high.where(high > close.shift(1), close.shift(1)) - low.where(low < close.shift(1), close.shift(1)))

def ATR(TR: pd.Series, atr_length: int) -> pd.Series:
    return TR.rolling(window=atr_length).mean()

# crossover function
def CO(a: pd.Series, b: pd.Series) -> pd.Series:
    return (a > b) & (a.shift(1) < b.shift(1))



def UTBot(close, high, low, key_valueBuy, atr_lengthBuy, key_valueSell, atr_lengthSell) -> pd.Series:
    lossThresholdBuy = key_valueBuy * ATR(TR(close, high, low), atr_lengthBuy)
    trailingStopBuy = 0 * lossThresholdBuy
    lossThresholdSell = key_valueSell * ATR(TR(close, high, low), atr_lengthSell)
    trailingStopSell = 0 * lossThresholdSell

    for i in range(1, len(trailingStopBuy)):

        # the previous trailing stop is lower than the current and previous close
        if (close[i] > trailingStopBuy[i - 1]) & (close[i - 1] > trailingStopBuy[i - 1]):
            trailingStopBuy[i] = max(trailingStopBuy[i - 1], close[i] - lossThresholdBuy[i])
        # the previous trailing stop is higher than the current and previous close
        elif (close[i] < trailingStopBuy[i - 1]) & (close[i - 1] < trailingStopBuy[i - 1]):
            trailingStopBuy[i] = min(trailingStopBuy[i - 1], close[i] - lossThresholdBuy[i])
        # the previous trailing stop is lower than the current close and higher than the previous close
        elif (close[i] > trailingStopBuy[i - 1]):
            trailingStopBuy[i] = close[i] - lossThresholdBuy[i]
        # the previous trailing stop is higher than the current close and lower than the previous close
        else:
            trailingStopBuy[i] = close[i] + lossThresholdBuy[i]

    above = CO(close, trailingStopBuy)
    
    for i in range(1, len(trailingStopSell)):

        # the previous trailing stop is lower than the current and previous close
        if (close[i] > trailingStopSell[i - 1]) & (close[i - 1] > trailingStopSell[i - 1]):
            trailingStopSell[i] = max(trailingStopSell[i - 1], close[i] - lossThresholdSell[i])
        # the previous trailing stop is higher than the current and previous close
        elif (close[i] < trailingStopSell[i - 1]) & (close[i - 1] < trailingStopSell[i - 1]):
            trailingStopSell[i] = min(trailingStopSell[i - 1], close[i] - lossThresholdSell[i])
        # the previous trailing stop is lower than the current close and higher than the previous close
        elif (close[i] > trailingStopSell[i - 1]):
            trailingStopSell[i] = close[i] - lossThresholdSell[i]
        # the previous trailing stop is higher than the current close and lower than the previous close
        else:
            trailingStopBuy[i] = close[i] + lossThresholdSell[i]
    
    below = CO(trailingStopSell, close)


    Action = np.select([above, below], [ActionType.BUY, ActionType.SELL], default=ActionType.DONOTHING)
    

    # check if works properly!!!!!!
    return Action

# smoothing function
def SmoothSrs(srs, smoothing_f):
    smoothed_srs = srs.copy()
    for i in range(1, len(smoothed_srs)):
        if np.isnan(smoothed_srs[i-1]):
            smoothed_srs[i] = srs[i]
        else:
            smoothed_srs[i] = smoothed_srs[i-1] + smoothing_f * (srs[i] - smoothed_srs[i-1])
    return smoothed_srs

# normalization function with smoothing
def normNsmooth(srs, stc_length, smoothing_factor):
    # finding the lowest and highest range
    lowest = srs.rolling(stc_length).min()
    highestRange = srs.rolling(stc_length).max() - lowest
    
    # normalizing srs
    normalizedsrs = srs.copy()
    normalizedsrs[highestRange > 0] = ((srs - lowest) / highestRange * 100)*(highestRange > 0)
    normalizedsrs[highestRange <= 0] = np.nan
    normalizedsrs.fillna(method = 'ffill', inplace = True)

    # smoothing the srs
    return SmoothSrs(normalizedsrs, smoothing_factor)

# complete function for calculating osciallatior
def STCosi(srs, fast_length, slow_length, stc_length, smoothing_factor = 0.5):
    
    # ema calculation for fast and slow length's
    fast_ema = srs.ewm(span = fast_length).mean()
    slow_ema = srs.ewm(span = slow_length).mean()

    # MacdDiff calculation
    MacdDiff = fast_ema - slow_ema

    smoothedMacd = normNsmooth(MacdDiff, stc_length, smoothing_factor)
    FinalSTC = normNsmooth(smoothedMacd, stc_length, smoothing_factor)
    
    return FinalSTC

In [11]:
def smooth_srs(srs, smoothing_f):
    smoothed_srs = []
    smoothed_srs[0] = srs[0]

    for i in range(1, len(srs)):
        if pd.isna(smoothed_srs[i - 1]):
            smoothed_srs[i] = srs[i]

        else:
            smoothed_srs[i] = smoothed_srs[i-1] + smoothing_f * (srs[i] - smoothed_srs[i-1])
    return smoothed_srs

In [12]:
def NormalizeSmoothSrs(series, window_length, smoothing_f):
    lowest = series.rolling(window_length).min()
    highest_range = series.rolling(window_length).max() - lowest

    normalized_series = series
    if(highest_range > 0):
        normalized_value = (series - lowest) / highest_range * 100
    else:
        normalized_value = np.nan
    normalized_series.ffill(inplace=True)

    smoothed_series = smooth_srs(normalized_series, smoothing_f)
    return smoothed_series

In [13]:
def STC(close, stc_length, fast_length, slow_length, smoothing_factor=0.5):
    macd_diff = MacdDiff(close, fast_length, slow_length)
    normalized_macd = NormalizeSmoothSrs(macd_diff, stc_length, smoothing_factor)
    final_stc = NormalizeSmoothSrs(normalized_macd, stc_length, smoothing_factor)
    return final_stc

In [14]:
def calc_ut_bot(df):
    df = calc_TR(df)
    df = calc_ATR(df, 15)
    df = calc_UTBot(df, key_value=1, atr_length=15)
    return df

In [15]:
ut_bot = calc_ut_bot(df.copy())
ut_bot

C:\Users\tomge\AppData\Local\Temp\ipykernel_26344\2801935451.py:19: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['trailing_stop'][i] = df['close'][i] + df['loss_threshold'][i]
C:\Users\tomge\AppData\Local\Temp\ipykernel_26344\2801935451.

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,ignore,prev_close,TR,ATR,loss_threshold,trailing_stop,crossover_above,crossover_below,buy,sell
0,2024-01-01 00:00:00+02:00,42559.0,42613.0,42466.1,42608.9,2926.528,2024-01-01 00:29:59.999000+02:00,1.245095e+08,31338,1320.000,...,0.0,NaN,146.9,NaN,NaN,NaN,False,False,0,0
1,2024-01-01 00:30:00+02:00,42608.9,42629.5,42111.9,42294.8,9025.818,2024-01-01 00:59:59.999000+02:00,3.821358e+08,89174,3701.256,...,0.0,42608.9,517.6,NaN,NaN,NaN,False,False,0,0
2,2024-01-01 01:00:00+02:00,42294.8,42380.1,42083.1,42211.2,8760.876,2024-01-01 01:29:59.999000+02:00,3.699976e+08,82898,3594.587,...,0.0,42294.8,297.0,NaN,NaN,NaN,False,False,0,0
3,2024-01-01 01:30:00+02:00,42211.3,42315.6,42180.8,42314.0,2915.589,2024-01-01 01:59:59.999000+02:00,1.232041e+08,33344,1624.211,...,0.0,42211.2,134.8,NaN,NaN,NaN,False,False,0,0
4,2024-01-01 02:00:00+02:00,42314.0,42603.2,42289.6,42458.5,5940.016,2024-01-01 02:29:59.999000+02:00,2.522419e+08,58673,3456.112,...,0.0,42314.0,313.6,NaN,NaN,NaN,False,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,2024-01-06 22:00:00+02:00,43903.6,43960.7,43645.0,43855.7,5595.282,2024-01-06 22:29:59.999000+02:00,2.450022e+08,62110,2374.617,...,0.0,43903.6,315.7,145.360000,145.360000,44001.060000,False,True,0,1
285,2024-01-06 22:30:00+02:00,43855.7,43897.4,43775.3,43816.4,1752.555,2024-01-06 22:59:59.999000+02:00,7.683397e+07,26867,707.315,...,0.0,43855.7,122.1,143.306667,143.306667,43673.093333,True,False,1,0
286,2024-01-06 23:00:00+02:00,43816.3,43901.7,43712.1,43868.0,2106.253,2024-01-06 23:29:59.999000+02:00,9.226863e+07,32340,1048.501,...,0.0,43816.4,189.6,149.406667,149.406667,43718.593333,False,False,0,0
287,2024-01-06 23:30:00+02:00,43868.0,43930.4,43798.1,43881.3,2436.663,2024-01-06 23:59:59.999000+02:00,1.069033e+08,34767,1351.523,...,0.0,43868.0,132.3,146.566667,146.566667,43734.733333,False,False,0,0


In [21]:
ut_bot.dtypes

open_time                       datetime64[ms, Asia/Jerusalem]
open                                                   float64
high                                                   float64
low                                                    float64
close                                                  float64
volume                                                 float64
close_time                      datetime64[ms, Asia/Jerusalem]
quote_asset_volume                                     float64
number_of_trades                                         int64
taker_buy_base_asset_volume                            float64
taker_buy_quote_asset_volume                           float64
ignore                                                 float64
prev_close                                             float64
TR                                                     float64
ATR                                                    float64
loss_threshold                                         

In [93]:
df['UTBot_indicator'] = UTBot(df.close, df.high, df.low, 2, 1, 2, 300)
df['STC_indicator'] = STCosi(df.close, 23, 50, 10, 0.5)
df

C:\Users\tomge\AppData\Local\Temp\ipykernel_26344\283304217.py:80: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normalizedsrs.fillna(method = 'ffill', inplace = True)
C:\Users\tomge\AppData\Local\Temp\ipykernel_26344\283304217.py:80: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  normalizedsrs.fillna(method = 'ffill', inplace = True)


,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,UTBot_indicator,STC_indicator
0,2024-01-01 00:00:00+02:00,42559.0,42613.0,42466.1,42608.9,2926.528,2024-01-01 00:29:59.999000+02:00,1.245095e+08,31338,1320.000,5.617103e+07,0.0,ActionType.DONOTHING,0.000000
1,2024-01-01 00:30:00+02:00,42608.9,42629.5,42111.9,42294.8,9025.818,2024-01-01 00:59:59.999000+02:00,3.821358e+08,89174,3701.256,1.566922e+08,0.0,ActionType.DONOTHING,-0.921815
2,2024-01-01 01:00:00+02:00,42294.8,42380.1,42083.1,42211.2,8760.876,2024-01-01 01:29:59.999000+02:00,3.699976e+08,82898,3594.587,1.518230e+08,0.0,ActionType.BUY,-2.457324
3,2024-01-01 01:30:00+02:00,42211.3,42315.6,42180.8,42314.0,2915.589,2024-01-01 01:59:59.999000+02:00,1.232041e+08,33344,1624.211,6.863622e+07,0.0,ActionType.DONOTHING,-3.566754
4,2024-01-01 02:00:00+02:00,42314.0,42603.2,42289.6,42458.5,5940.016,2024-01-01 02:29:59.999000+02:00,2.522419e+08,58673,3456.112,1.467360e+08,0.0,ActionType.DONOTHING,-3.447998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,2024-01-06 22:00:00+02:00,43903.6,43960.7,43645.0,43855.7,5595.282,2024-01-06 22:29:59.999000+02:00,2.450022e+08,62110,2374.617,1.040008e+08,0.0,ActionType.DONOTHING,96.573092
285,2024-01-06 22:30:00+02:00,43855.7,43897.4,43775.3,43816.4,1752.555,2024-01-06 22:59:59.999000+02:00,7.683397e+07,26867,707.315,3.100777e+07,0.0,ActionType.DONOTHING,49.120926
286,2024-01-06 23:00:00+02:00,43816.3,43901.7,43712.1,43868.0,2106.253,2024-01-06 23:29:59.999000+02:00,9.226863e+07,32340,1048.501,4.593668e+07,0.0,ActionType.DONOTHING,24.560463
287,2024-01-06 23:30:00+02:00,43868.0,43930.4,43798.1,43881.3,2436.663,2024-01-06 23:59:59.999000+02:00,1.069033e+08,34767,1351.523,5.930249e+07,0.0,ActionType.DONOTHING,12.280232
